In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Function to load and preprocess fingerprint images from the updated folder structure
def load_fingerprint_dataset(base_path):
    images = []
    labels = []
    label_map = {}

    for person_id in range(1, 46):  # 45 people
        person_folder = os.path.join(base_path, str(person_id), 'Fingerprint')
        
        # Loop through all 10 fingerprint images for this person
        for filename in os.listdir(person_folder):
            if filename.endswith('.BMP'):
                image_path = os.path.join(person_folder, filename)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, (128, 128))  # Resize for uniform input
                image = image / 255.0  # Normalize to [0, 1]
                images.append(image)

                # Use the filename (e.g., "1_M_Left_index_finger") as the label
                label = f"person_{person_id}_{filename.split('.')[0]}"
                labels.append(label)

                # Store the mapping from label to person ID
                label_map[label] = person_id

    images = np.array(images)
    labels = np.array(labels)
    return images, labels, label_map

# Example usage
base_path = "/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET"
X, y, label_map = load_fingerprint_dataset(base_path)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

ImportError: initialization failed

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Lambda, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# Building the base model to learn fingerprint features
def build_base_model(input_shape):
    inputs = Input(input_shape)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    outputs = Dense(128, activation='relu')(x)
    
    return Model(inputs, outputs)

# Custom Lambda layer to calculate the L1 distance between embeddings
def l1_distance(vectors):
    x, y = vectors
    return K.abs(x - y)

# Building the Siamese network for fingerprint matching
def build_siamese_network(input_shape):
    base_model = build_base_model(input_shape)
    
    input_a = Input(input_shape)
    input_b = Input(input_shape)
    
    feats_a = base_model(input_a)
    feats_b = base_model(input_b)
    
    distance = Lambda(l1_distance)([feats_a, feats_b])
    
    outputs = Dense(1, activation='sigmoid')(distance)
    
    model = Model([input_a, input_b], outputs)
    
    return model

# Example usage
input_shape = (128, 128, 1)  # For grayscale fingerprint images
siamese_model = build_siamese_network(input_shape)

# Compile the model
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

siamese_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 128)       │ 536,997,6… │ input_layer_1[0]… │
│ (Functional)        │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 128)       │          0 │ functional_1[0][… │
│                     │                   │            │ functional_1[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │        129 │ lambda[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 536,997,761 (2.00 GB)

 Trainable params: 536,997,313 (2.00 GB)

 Non-trainable params: 448 (1.75 KB)

In [4]:
import numpy as np

def create_pairs(images, labels, label_map):
    pairs = []
    labels_out = []

    # Get unique labels
    unique_labels = np.unique(labels)
    
    for label in unique_labels:
        # Get all indices for the current person (label)
        indices = [i for i, l in enumerate(labels) if l == label]
        current_image = images[indices[0]]  # Select one image to start
        
        # Ensure there are at least two images for positive pairing
        if len(indices) > 1:
            # Create a positive pair (same person, different fingers)
            positive_index = np.random.choice([idx for idx in indices if idx != indices[0]])
            positive_image = images[positive_index]
            pairs += [[current_image, positive_image]]
            labels_out += [1]  # Positive pair

        # Find a negative example (different person)
        other_label = np.random.choice([l for l in unique_labels if l != label])
        other_indices = [i for i, l in enumerate(labels) if l == other_label]
        negative_image = images[np.random.choice(other_indices)]
        
        pairs += [[current_image, negative_image]]
        labels_out += [0]  # Negative pair

    return np.array(pairs), np.array(labels_out)

# Example usage
pairs, labels = create_pairs(X, y, label_map)

In [5]:
from sklearn.model_selection import train_test_split

# Split pairs and labels into training and validation sets
X_train_pairs, X_val_pairs, y_train_pairs, y_val_pairs = train_test_split(pairs, labels, test_size=0.2, random_state=42)

# Train the model
history = siamese_model.fit(
    [X_train_pairs[:, 0], X_train_pairs[:, 1]], y_train_pairs,
    validation_data=([X_val_pairs[:, 0], X_val_pairs[:, 1]], y_val_pairs),
    batch_size=8,
    epochs=30
)

Epoch 1/30


I0000 00:00:1726593597.009101     100 service.cc:145] XLA service 0x7aa5c8013660 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726593597.009168     100 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1726593597.009175     100 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1726593633.518675     100 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


45/45 ━━━━━━━━━━━━━━━━━━━━ 54s 284ms/step - accuracy: 1.0000 - loss: 0.0318 - val_accuracy: 1.0000 - val_loss: 1.9106e-21
Epoch 2/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 9s 209ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 9s 210ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 10s 211ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 10s 213ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 10s 215ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 10s 215ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 10s 217ms/step - acc

In [9]:
import os

# Preprocess a fingerprint image before comparison
def preprocess_image(image_path):
    # Check if the file exists
    if not os.path.exists(image_path):
        print(f"File not found: {image_path}")
        return None
    
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if image is None:
        print(f"Error reading the image: {image_path}")
        return None
    
    image = cv2.resize(image, (128, 128))
    image = image / 255.0
    image = np.expand_dims(image, axis=-1)  # Add channel dimension for model input
    return image

In [21]:
import numpy as np

def is_authorized(input_image_path, reference_images_paths, model, threshold=0.45):
    input_image = preprocess_image(input_image_path)
    
    if input_image is None:
        print(f"Failed to load input image: {input_image_path}")
        return "Not Authorized"
    
    input_image = np.expand_dims(input_image, axis=0)  # Expand dimensions for batch processing
    
    for ref_image_path in reference_images_paths:
        ref_image = preprocess_image(ref_image_path)
        
        if ref_image is None:
            print(f"Skipping reference image: {ref_image_path}")
            continue  # Skip if the reference image couldn't be loaded
        
        ref_image = np.expand_dims(ref_image, axis=0)  # Expand dimensions for batch processing
        
        # Predict similarity between input image and reference image
        score = model.predict([input_image, ref_image])[0][0]
        print(f"Comparing with {ref_image_path}, similarity score: {score}")
        
        # Check if the similarity score exceeds the threshold
        if score >= threshold:
            print(f"Authorized based on reference: {ref_image_path} with score {score}")
            return "Authorized"
    
    print("No match found, not authorized")
    return "Not Authorized"


# Example usage
reference_images = [
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Left_index_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Left_little_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Left_middle_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Left_ring_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Left_thumb_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Right_index_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Right_little_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Right_middle_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Right_ring_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Right_thumb_finger.BMP',
    
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/2/Fingerprint/2__F_Left_index_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/2/Fingerprint/2__F_Left_little_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/2/Fingerprint/2__F_Left_middle_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/2/Fingerprint/2__F_Left_ring_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/2/Fingerprint/2__F_Left_thumb_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/2/Fingerprint/2__F_Right_index_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/2/Fingerprint/2__F_Right_little_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/2/Fingerprint/2__F_Right_middle_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/2/Fingerprint/2__F_Right_ring_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/2/Fingerprint/2__F_Right_thumb_finger.BMP',
    
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/3/Fingerprint/3__M_Left_index_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/3/Fingerprint/3__M_Left_little_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/3/Fingerprint/3__M_Left_middle_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/3/Fingerprint/3__M_Left_ring_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/3/Fingerprint/3__M_Left_thumb_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/3/Fingerprint/3__M_Right_index_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/3/Fingerprint/3__M_Right_little_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/3/Fingerprint/3__M_Right_middle_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/3/Fingerprint/3__M_Right_ring_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/3/Fingerprint/3__M_Right_thumb_finger.BMP',
    
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/4/Fingerprint/4__M_Left_index_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/4/Fingerprint/4__M_Left_little_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/4/Fingerprint/4__M_Left_middle_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/4/Fingerprint/4__M_Left_ring_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/4/Fingerprint/4__M_Left_thumb_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/4/Fingerprint/4__M_Right_index_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/4/Fingerprint/4__M_Right_little_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/4/Fingerprint/4__M_Right_middle_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/4/Fingerprint/4__M_Right_ring_finger.BMP',
    '/kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/4/Fingerprint/4__M_Right_thumb_finger.BMP',
    
]
result = is_authorized('/kaggle/input/test-img/1.jpg', reference_images, siamese_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Comparing with /kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Left_index_finger.BMP, similarity score: 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Comparing with /kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Left_little_finger.BMP, similarity score: 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Comparing with /kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Left_middle_finger.BMP, similarity score: 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Comparing with /kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Left_ring_finger.BMP, similarity score: 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Comparing with /kaggle/input/multimodal-iris-fingerprint-biometric-data/IRIS and FINGERPRINT DATASET/1/Fingerprint/1__M_Left_thumb_finger.BMP, simila

In [3]:
pip install speechbrain

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import torchaudio
from speechbrain.pretrained import SpeakerRecognition
from torch.nn import functional as F

# Load pretrained Speaker Recognition model from SpeechBrain
model = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_model")

# Load master speech (reference speech)
master_file = "/kaggle/input/audio-data/audio_files/master_audio_main.wav"
master_signal, _ = torchaudio.load(master_file)
master_embedding = model.encode_batch(master_signal)

# Function to get embedding for new speech
def get_embedding(file_path):
    signal, _ = torchaudio.load(file_path)
    return model.encode_batch(signal)

# Function to reduce embeddings (average across time or batch dimension)
def reduce_embedding(embedding):
    # Average across time (dim=2) and batch (dim=0)
    return torch.mean(embedding, dim=(0, 2))

In [ ]:
# Function to calculate similarity
def calculate_similarity(embedding1, embedding2):
    # Reduce both embeddings to 1D vectors
    reduced_embedding1 = reduce_embedding(embedding1)
    reduced_embedding2 = reduce_embedding(embedding2)
    
    # Calculate cosine similarity
    return F.cosine_similarity(reduced_embedding1, reduced_embedding2, dim=0)

# Compare master speech with a new speech
new_speech_file = "/kaggle/input/audio-data/audio_files/speech.wav"
new_embedding = get_embedding(new_speech_file)

similarity_score = calculate_similarity(master_embedding, new_embedding)
print(f"Similarity score: {similarity_score.item()}")

In [ ]:
threshold = 0.85  # Set this based on testing different speeches
if similarity_score > threshold:
    print("You are authorized to enter.")
else:
    print("Authorization failed.")